In [1]:
import numpy as np
import pandas as pd
import os

In [3]:
PATH = 'C:/Workspace/python/Data_Science/dataA/ProDS특강/'
file_list = os.listdir(PATH)
print(file_list)

['Dataset.zip', 'Dataset_01.csv', 'Dataset_02.csv', 'Dataset_03.csv', 'Dataset_04.csv', 'Dataset_05.csv', 'Dataset_05_item_list.csv', 'Dataset_05_Mart_POS.csv', 'Dataset_06.csv', 'Dataset_07.csv', 'Dataset_08.csv', 'Dataset_09.csv', 'Dataset_10.csv', 'Dataset_11.csv', 'Dataset_12.csv', 'Dataset_13_test.csv', 'Dataset_13_train.csv', 'Dataset_14.csv', 'Dataset_15_item_list.csv', 'Dataset_15_Mart_POS.csv']


In [14]:
print(file_list[1])
df_01 = pd.read_csv(PATH + file_list[1])
df_01.head()

Dataset_01.csv


,TV,Radio,Social_Media,Influencer,Sales
0,16.0,6.566231,2.907983,Mega,54.732757
1,13.0,9.237765,2.409567,Mega,46.677897
2,41.0,15.886446,2.913410,Mega,150.177829
3,83.0,30.020028,6.922304,Mega,298.246340
4,15.0,8.437408,1.405998,Micro,56.594181


R-squared: 0.6511

R제곱
모형 적합도(혹은 설명력)
dist의 분산을 speed가 약 65%를 설명한다
각 사례마다 dist에 차이가 있다.

Adj. R-squared
독립변수가 여러 개인 다중회귀분석에서 사용
독립변수의 개수와 표본의 크기를 고려하여 R-squared를 보정
서로 다른 모형을 비교할 때는 이 지표가 높은 쪽은 선택한다


F-statistic, Prob(F-statistic)
회귀모형에 대한 (통계적) 유의미성 검증 결과, 유의미함 (p < 0.05)
즉, 이 모형은 주어진 표본 뿐 아니라 모집단에서도 의미있는 모형이라 할 수 있음


로그 우도: 종속변수가 정규분포라 가정했을 때 그 우도
로그우도도 R제곱과 마찬가지로 독립변수가 많아지면 증가한다

AIC, BIC: 로그우도를 독립변수의 수로 보정한 값 (작을 수록 좋다)

추정치의 표 중간의 P(>|t|)는 모집단에서 계수가 0일 때, 현재와 같은 크기의 표본에서 이러한 계수가 추정될 확률인 p값을 나타낸다. 이 확률이 매우 작다는 것은, 모집단에서 speed의 계수가 정확히 3.9324는 아니더라도 현재의 표본과 비슷하게 0보다 큰 어떤 범위에 있을 가능성이 높다는 것을 의미한다. 보통 5%와 같은 유의수준을 정하여 p값이 그보다 작으면(p < 0.05), "통계적으로 유의미하다"라고 한다.
즉, speed가 증가할 때 기대되는 dist의 변화는 유의수준 5%에서 통계적으로 유의미하다.

In [75]:
# 1. 데이터 세트 내에 총 결측값의 개수는 몇 개인가? (답안 예시) 23

q1 = df_01.copy()
q1_out = q1.isnull().sum().sum()
print(f'1번 답: {q1_out}')
# (q1.isna().sum(axis=1)>=1).sum() 결측값이 있는 행의 수

# 2. TV, Radio, Social Media 등 세 가지 다른 마케팅 채널의 예산과 매출액과의 상관분석을
# 통하여 각 채널이 매출에 어느 정도 연관이 있는지 알아보고자 한다. 
# - 매출액과 가장 강한 상관관계를 가지고 있는 채널의 상관계수를 소수점 5번째
# 자리에서 반올림하여 소수점 넷째 자리까지 기술하시오. (답안 예시) 0.1234

q2 = q1.drop(['Influencer'], axis = 1).copy()
q2_tab = q2.corr().drop(['TV', 'Radio', 'Social_Media'], axis = 1).copy()
q2_out = q2_tab.drop(['Sales']) 
print(f'2번답 : {q2_out.Sales.idxmax()} : {round(q2_out.Sales.max(), 5)}') # 반올림
print(f'2번답 : {q2_out.Sales.idxmax()} : {np.ceil(q2_out.Sales.max()*10000) / 10000}') # 올림
print(f'2번답 : {q2_out.Sales.idxmax()} : {np.floor(q2_out.Sales.max()*10000) / 10000}') # 내림

# 3. 매출액을 종속변수, TV, Radio, Social Media의 예산을 독립변수로 하여 회귀분석을
# 수행하였을 때, 세 개의 독립변수의 회귀계수를 큰 것에서부터 작은 것 순으로
# 기술하시오. 
# - 분석 시 결측치가 포함된 행은 제거한 후 진행하며, 회귀계수는 소수점 넷째 자리
# 이하는 버리고 소수점 셋째 자리까지 기술하시오. (답안 예시) 0.123

from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols 
from statsmodels.api import OLS, add_constant

q3 = df_01.copy()
q3 = q3.dropna().reset_index(drop = True)
q3 = q3.drop(['Influencer'], axis = 1)
lm = LinearRegression()
lm.fit(q3[['TV', 'Radio', 'Social_Media']], q3.Sales)
print(lm.coef_)

# ols = Ordinary least square(최소 자승법 : 잔차제곱합을 최소화하는 가중치 벡터를 구하는 방법)
# Df Model : 독립변수의 개수 // Df(자유도) residuals : 전체 표본개수에서 독립변수, 종속변수의 개수를 뺀 것
# intercept coef: 절편 // 독립변수 coef : 기울기
# Dep.Variable : Dependent variable = 종속변수 // Model : 모델링 방법
# R squared : 결정계수 - 전체 데이터중 해당 모델이 설명할 수 있는 데이터의 비율 = 설명력 / 값이 크면 예측이 정확(0~1)
# F-statistics : F 통계량 - 값이 크면 선형이 잘 안그려짐(0 ~ inf)
# Prob : F 통계량에 해당하는 P-value
x_list = ['TV', 'Radio', 'Social_Media']
formula = 'Sales~' + '+'.join(x_list)
model = OLS.from_formula(formula, data = q3)
result = model.fit()
print(result.params)
result.summary()

1번 답: 26
2번답 : TV : 0.9995
2번답 : TV : 0.9995
2번답 : TV : 0.9994
[ 3.56256963 -0.00397039  0.00496402]
Intercept      -0.133963
TV              3.562570
Radio          -0.003970
Social_Media    0.004964
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.505e+06
Date:                Tue, 30 Nov 2021   Prob (F-statistic):               0.00
Time:                        01:02:51   Log-Likelihood:                -11366.
No. Observations:                4546   AIC:                         2.274e+04
Df Residuals:                    4542   BIC:                         2.277e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.1340      0.103     -1.303      0.193      -0.336       0.068
TV               3.5626      0.003   1051.118      0.000       3.556       3.569
Radio           -0.0040      0.010     -0.406      0.685      -0.023       0.015
Social_Media     0.0050      0.025      0.199      0.842      -0.044       0.054
==============================================================================
Omnibus:                        0.056   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.972   Jarque-Bera (JB):                0.034
Skew:                          -0.001   Prob(JB):                        0.983
Kurtosis:                       3.013   Cond. No.                         149.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""